<a href="https://colab.research.google.com/github/fatday/STATS-305B-HW4-Group/blob/main/Part2_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2 Code Set up

In [1]:
# torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from tqdm import tqdm
import requests
import os
import json
import re
from collections import Counter
import sentencepiece as spm

torch.manual_seed(305)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

We set default values for some global hyperparameters, but feel free to change these during development as needed.

In [2]:
# Global hyperparameters
SMALL_ITERS = 1000
LARGE_ITERS = 2000
EVAL_ITERS = 100
CONTEXT_WINDOW_SIZE = 256

## Preprocessing

In [3]:
# download the tiny shakespeare dataset
input_file_path = 'input.txt'

if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

length of dataset in characters: 1,115,394


## Tokenizer and its Sub-Classes

In [4]:
class Tokenizer:
    def __init__(self, data):
        self.data = data
        self.stoi = {}
        self.itos = {}
        self.vocab_size = 0
        self.train_data = None
        self.val_data = None

    def encode(self, s):
        return [self.stoi[c] for c in s]

    def decode(self, l):
        return ''.join([self.itos[i] for i in l])


    def get_batch(self, split, context_window_size, device, batch_size=32):

        current_data = self.train_data if split == 'train' else self.val_data
        ix = torch.randint(len(current_data) - context_window_size, (batch_size,))
        x = torch.stack([current_data[i:i+context_window_size] for i in ix])
        y = torch.stack([current_data[i+1:i+context_window_size+1] for i in ix])
        x = x.to(device)
        y = y.to(device)
        return x, y

    @torch.no_grad()
    def estimate_loss(self, model, eval_iters, context_window_size, device):
        # estimate loss, perplexity, character_level perplexity
        lossout, perplexity, char_perplexity = {}, {}, {}
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            perplexities = torch.zeros(eval_iters)
            nll = 0
            char_cnt = 0
            for k in range(eval_iters):
                X, Y = self.get_batch(split, context_window_size, device)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
                perplexities[k] = torch.exp(loss).item()
                nll += loss.item()
                char_cnt += len(self.decode(Y[0].tolist()))

            char_perplexity[split] = torch.exp(torch.tensor(nll / char_cnt)).item()
            lossout[split] = losses.mean()
            perplexity[split] = perplexities.mean()
        return lossout, perplexity, char_perplexity


In [5]:
class CharacterTokenizer(Tokenizer):
    def __init__(self, data):
        self.data = data
        self.chars = sorted(list(set(data)))
        self.vocab_size = len(self.chars)

        self.train_chars = data[:int(len(data)*0.9)]
        self.val_chars = data[int(len(data)*0.9):]

        self.stoi = { ch:i for i,ch in enumerate(self.chars) }
        self.itos = { i:ch for i,ch in enumerate(self.chars) }

        self.train_data = torch.tensor(self.encode(self.train_chars))
        self.val_data = torch.tensor(self.encode(self.val_chars))

In [6]:
class SimpleWordTokenizer(Tokenizer):
    def __init__(self, data):
        self.data = data
        self.data_words = re.findall(r'\w+|\s+|[^\w\s]', data)
        self.vocab_set = set(self.data_words).union(set(data))
        self.vocab_size = len(self.vocab_set)

        self.stoi = {ch:i for i,ch in enumerate(self.vocab_set)}
        self.itos = {i:ch for i,ch in enumerate(self.vocab_set)}

        self.train_words = self.data_words[:int(len(self.data_words)*0.9)]
        self.val_words = self.data_words[int(len(self.data_words)*0.9):]

        self.train_data = torch.tensor(self.encode(self.train_words))
        self.val_data = torch.tensor(self.encode(self.val_words))

In [7]:
class BPETokenizer(Tokenizer):
    def __init__(self, data):
        self.data = data
        # BPE
        spm.SentencePieceTrainer.Train(input="input.txt",
        model_prefix="tokenizer",
        vocab_size=3000,
        model_type="bpe",
        normalization_rule_name="identity",
        character_coverage=1.0,
        add_dummy_prefix=False,
        user_defined_symbols = ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?'])

        self.sp = spm.SentencePieceProcessor(model_file="tokenizer.model")
        self.vocab_size = self.sp.vocab_size()


        self.train_chars = data[:int(len(data)*0.9)]
        self.val_chars = data[int(len(data)*0.9):]
        self.train_data = torch.tensor(self.encode(self.train_chars))
        self.val_data = torch.tensor(self.encode(self.val_chars))

    def encode(self, s):
      return self.sp.encode(s, out_type=int)

    def decode(self, l):
      return self.sp.decode(l)

## Original Head

In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, context_window_size, embed_size=384):
        """
        Args:
          head_size: int, size of the head embedding dimension (K)
          context_window_size: int, number of tokens considered in the past for attention (T)
          embed_size: int, size of the token embedding dimension (D)
        """
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(embed_size, head_size, bias=False)
        self.query = nn.Linear(embed_size, head_size, bias=False)
        self.value = nn.Linear(embed_size, embed_size, bias=False)

        # not a param of the model, so registered as a buffer
        self.register_buffer('tril', torch.tril(
            torch.ones(context_window_size, context_window_size)))

    def forward(self, x):
        """
        Args:
          x: (B,T,D) tensor of token embeddings

        Returns:
          (B,T,D) tensor of attention-weighted token embeddings
        """
        # TODO: your code here
        B,T,D = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2,-1) * self.head_size**-0.5
        #tril = torch.tril(torch.ones(T, T, device=x.device))
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)  ## wei.shape:
        out = wei @ v
        return out

In [9]:
class SingleHeadedAttentionLM(nn.Module):

    def __init__(self, vocab_size, context_window_size, head_size, embed_size=384):
      """
      Args:
        vocab_size: int, size of the vocabulary (V)
        context_window_size: int, number of tokens considered in the past for attention (T)
        head_size: int, size of the head embedding dimension (K)
        embed_size: int, size of the token embedding dimension (D)
      """
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
      self.position_embedding_table = nn.Embedding(context_window_size, embed_size)
      self.context_window_size = context_window_size

      # TODO: your code below
      self.atten_head = Head(head_size, context_window_size)
      self.lm_head = nn.Linear(embed_size, vocab_size)
      self.context_window_size = context_window_size

    def forward(self, token_ids, targets=None):
        """
        Args:
          token_ids: (B, T) token ids that make up the context (batch has size B, each entry
                     in the batch has length T)
          targets: (B, T) token ids corresponding to the target of each context in token_ids

        Returns:
          logits: (B, T, V) logits[b,t] gives the length V vector of logits for the next token
                   prediction in string b up to t tokens
          loss: scalar, negative log likelihood of target given context
        """
        B, T = token_ids.shape # (batch size, length)
        tok_emb = self.token_embedding_table(token_ids) # (B,T,D)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,D)
        x = tok_emb + pos_emb # (B,T,D)
        x = self.atten_head(x) # (B,T,D)
        logits = self.lm_head(x) # (B,T,V)

        # TODO: your code here
        B, T, V = logits.shape
        logits = logits.view(B*T, V)
        if targets is None:
            loss = None
        else:
            targets = targets.view(B*T)
            loss = -torch.mean(torch.log(F.softmax(logits, dim=1)[torch.arange(B*T), targets]))
        return logits, loss

    @torch.no_grad()
    def generate(self, token_ids, max_new_tokens):
        """
        Args:
          token_ids: (B, T) tensor of token ids to provide as context
          max_new_tokens: int, maximum number of new tokens to generate

        Returns:
          (B, T+max_new_tokens) tensor of context with new tokens appended
        """
        #TODO
        # your code below
        B, T = token_ids.shape
        new_token_sequences = torch.zeros((B, T+max_new_tokens), dtype=torch.long, device=token_ids.device)
        new_token_sequences[:, :T] = token_ids
        for t in range(max_new_tokens):
          input_tokens = new_token_sequences[:, max(0, T + t - self.context_window_size): T + t]
          logits, loss = self(input_tokens)
          logits = logits.view(B, min(T + t, self.context_window_size), -1)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          new_token = torch.multinomial(probs, num_samples=1)
          new_token_sequences[:, T + t] = new_token.squeeze(-1)
        return new_token_sequences

In [10]:

class MultiHeadAttention(nn.Module):
    def __init__(self, context_window_size, num_heads, embed_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = embed_size // num_heads  # 确保总维度匹配

        self.heads = nn.ModuleList(
            [Head(self.head_size, context_window_size, embed_size) for _ in range(num_heads)]
        )
        self.proj = nn.Linear(embed_size * num_heads, embed_size)  # 确保总维度匹配
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # 拼接 heads

        out = self.dropout(self.proj(out))  # 投影回 embed_size
        return out


In [11]:
class MultiHeadedAttentionLM(nn.Module):

    def __init__(self, vocab_size, context_window_size, embed_size=384, num_heads=6):
      super().__init__()
      self.head_size = embed_size // num_heads
      self.context_window_size = context_window_size
      # TODO: your code below
      self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
      self.position_embedding_table = nn.Embedding(context_window_size, embed_size)
      self.atten_heads = MultiHeadAttention(context_window_size, num_heads, embed_size)
      self.lm_head = nn.Linear(embed_size, vocab_size)


    def forward(self, token_ids, targets=None):
        """
        Args:
          token_ids: (B, T) token ids that make up the context (batch has size B, each entry in the
                     batch has length T)
          targets: (B, T) token ids corresponding to the target of each context in token_ids

        Returns:
          logits: (B, T, V), logits[b,t] gives the length V vector of logits for the next token
                  prediction in string b up to t tokens
          loss: scalar, negative log likelihood of target given context
        """
        # TODO: your code below
        B, T = token_ids.shape
        tok_emb = self.token_embedding_table(token_ids)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.atten_heads(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, -1)
            targets = targets.view(B*T)
            loss = -torch.mean(torch.log(F.softmax(logits, dim=1)[torch.arange(B*T), targets]))

        return logits, loss

    @torch.no_grad()
    def generate(self, token_ids, max_new_tokens):
        """
        Args:
          token_ids: (B, T) tensor of token ids to provide as context
          max_new_tokens: int, maximum number of new tokens to generate

        Returns:
          (B, T+max_new_tokens) tensor of context with new tokens appended
        """
        # TODO: your code below
        B, T = token_ids.shape
        new_token_sequences = torch.zeros((B, T+max_new_tokens), dtype=torch.long, device=token_ids.device)
        new_token_sequences[:, :T] = token_ids
        for t in range(max_new_tokens):
          input_tokens = new_token_sequences[:, max(0, T + t - self.context_window_size): T + t]
          logits, loss = self(input_tokens)
          logits = logits.view(B,min(T + t, self.context_window_size), -1)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          new_token = torch.multinomial(probs, num_samples=1)
          new_token_sequences[:, T + t] = new_token.squeeze(-1)
        return new_token_sequences

## RoE Head

In [12]:
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=2048):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer('inv_freq', inv_freq)
        self.max_seq_len_cached = max_position_embeddings
        self._set_cos_sin_cache(max_position_embeddings)

    def _set_cos_sin_cache(self, seq_len):
        t = torch.arange(seq_len, device=self.inv_freq.device)
        freqs = torch.einsum('i,j->ij', t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer('cos_cached', emb.cos()[None, None, :, :])
        self.register_buffer('sin_cached', emb.sin()[None, None, :, :])
        self.max_seq_len_cached = seq_len

    def forward(self, x, seq_len=None):
        # x: [batch, seq_len, dim]
        if seq_len > self.max_seq_len_cached:
            self._set_cos_sin_cache(seq_len)
        return self.cos_cached[:, :, :seq_len, ...], self.sin_cached[:, :, :seq_len, ...]

def rotate_half(x):
    x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, k, cos, sin):
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

# Modify the Head class to use RoPE
class HeadWithRoPE(nn.Module):
    def __init__(self, head_size, context_window_size, embed_size=384):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(embed_size, head_size, bias=False)
        self.query = nn.Linear(embed_size, head_size, bias=False)
        self.value = nn.Linear(embed_size, embed_size, bias=False)
        self.rope = RotaryEmbedding(head_size)
        self.register_buffer('tril', torch.tril(torch.ones(context_window_size, context_window_size)))

    def forward(self, x):
        B, T, D = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # Apply RoPE to queries and keys
        cos, sin = self.rope(x, seq_len=T)
        q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # Rest of the attention mechanism remains the same
        wei = q @ k.transpose(-2, -1) * self.head_size**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        out = wei @ v
        return out

In [13]:
class MultiHeadAttentionWithRoPE(nn.Module):
    def __init__(self, context_window_size, num_heads, embed_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = embed_size // num_heads

        # QKV projections
        self.q_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.k_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.v_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.out_proj = nn.Linear(embed_size, embed_size)

        # RoPE
        self.rope = RotaryEmbedding(self.head_size)
        # Causal mask
        self.register_buffer('mask', torch.tril(torch.ones(context_window_size, context_window_size)))

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, D = x.shape

        # Split heads
        q = self.q_proj(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)
        k = self.k_proj(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)
        v = self.v_proj(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)

        # Apply RoPE to queries and keys
        cos, sin = self.rope(x, seq_len=T)
        q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # Attention
        scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_size)
        scores = scores.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        # Combine heads
        out = (attn @ v).transpose(1, 2).contiguous().view(B, T, D)
        out = self.out_proj(out)
        return out

In [14]:
# run this cell to initialize this deep learning module that you should use in the code your write later
# you don't need to edit this layer
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity
        Given to you, you don't need to write any code here!
    """

    def __init__(self, embed_size, gelu = False, dropout = False):
        super().__init__()


        self.activation = nn.GELU() if gelu else nn.ReLU()
        layers = [nn.Linear(embed_size, 4 * embed_size), self.activation]
        if dropout:
            layers.append(nn.Dropout(0.2))

        layers.append(nn.Linear(4 * embed_size, embed_size))
        if dropout:
            layers.append(nn.Dropout(0.2))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

## Transformer and Blocks

In [15]:
class TransformerBlock(nn.Module):
    """ Transformer block: communication across sequence length, followed by communication across embedding space
        Uses multi-headed attention
    """

    def __init__(self, vocab_size, context_window_size, embed_size=384, num_heads=6, roe = False,
                  gelu = False, dropout = False, rmsnorm = False):
        super().__init__()
        self.ln1 = nn.RMSNorm(embed_size) if rmsnorm else nn.LayerNorm(embed_size)
        self.ln2 = nn.RMSNorm(embed_size) if rmsnorm else nn.LayerNorm(embed_size)

        # TODO: your code below
        self.feed_forward = FeedForward(embed_size, gelu = gelu, dropout = dropout)
        self.atten_heads = MultiHeadAttentionWithRoPE(context_window_size, num_heads, embed_size) if roe else MultiHeadAttention(context_window_size, num_heads, embed_size)

    def forward(self, x):
        x = x + self.atten_heads(self.ln1(x)) # communication over sequence length
        x = x + self.feed_forward(self.ln2(x)) # communication across embedding space
        return x

In [16]:
class TransformerLM(nn.Module):

    def __init__(self, vocab_size, context_window_size, embed_size=384, num_heads=6, n_layers=6,
                 roe = False, gelu = False, dropout = False, rmsnorm = False):
        """
          Args:
              vocab_size: int, number of tokens in the vocabulary (V)
              context_window_size: int, size of the context window (T)
              embed_size: int, embedding size (D)
              num_heads: int, number of heads (H)
              n_layers: int, number of layers (M)
        """
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        if not roe:
          self.position_embedding_table = nn.Embedding(context_window_size, embed_size)
        self.blocks = nn.Sequential(*[
            TransformerBlock(vocab_size,
                             context_window_size,
                             embed_size=embed_size,
                             num_heads=num_heads,
                             roe = roe,
                             gelu= gelu,
                             dropout = dropout,
                             rmsnorm = rmsnorm)
            for _ in range(n_layers)])

        # final layer norm
        self.ln_f = nn.RMSNorm(embed_size) if rmsnorm else nn.LayerNorm(embed_size)
        self.lm_head = nn.Linear(embed_size, vocab_size)
        self.context_window_size = context_window_size
        self.roe = roe
        self.gelu = gelu
        self.dropout = dropout
        self.rmsnorm = rmsnorm
        # good initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, token_ids, targets=None):
        """
        Agrgs:
            token_ids: tensor of integers, provides the contet, shape (B, T)
            targets: tensor of integers, provides the tokens we are preidcitng, shape (B, T)
        """
        B, T = token_ids.shape

        # token_ids and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(token_ids) # (B, T, D)
        if not self.roe:
          pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, D)
          x = tok_emb + pos_emb # (B, T, D)
        else:
          x = tok_emb

        # TODO: your code below
        logits = ...
        loss = ...

        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        B, T, V = logits.shape
        logits = logits.view(B*T, V)

        if targets is None:
            loss = None
        else:
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, token_ids, max_new_tokens):
        """
        Args:
            token_ids: tensor of integers forming the context, shape (B, T)
            max_new_tokens: int, max number of tokens to generate
        """
        # TOOD, your code below
        B, T = token_ids.shape
        context_length = self.position_embedding_table.num_embeddings
        new_token_sequences = torch.zeros((B, T+max_new_tokens), dtype=torch.long, device=token_ids.device)
        new_token_sequences[:, :T] = token_ids
        for t in range(max_new_tokens):
            input_tokens = new_token_sequences[:, max(0,T + t - context_length):T + t]
            logits, loss = self(input_tokens)
            logits = logits.view(B,min(T + t, self.context_window_size), -1)
            logits = logits[:, -1, :] # (B, V)
            probs = F.softmax(logits, dim=-1) # (B, V)
            new_token = torch.multinomial(probs, num_samples=1) # (B, 1)
            new_token_sequences[:, T + t] = new_token.squeeze(-1) # (B, T+1)
        return new_token_sequences

## Model & Tokenizer System -- (model, tokenizer) pair

For model training and text generation

In [17]:
class ModelTokenizerSystem:
  def __init__(self, model, tokenizer, device, optimizer_lr = 1e-4,
               optimizer_weight_decay = 1e-1, model_name = "None"):
    self.model = model
    self.tokenizer = tokenizer
    self.device = device
    self.optimizer = torch.optim.AdamW(model.parameters(),
                                       lr=optimizer_lr,
                                       weight_decay=optimizer_weight_decay)
    self.model_name = model_name
    self.weight_decay = optimizer_weight_decay

  def train(self, lr = 1e-3, eval_interval = 50, display_interval = 200,
            max_iter = 2000, early_stopping = True, min_delta = 0.001, verbose = True ):

    best_score = None
    self.loss_list = []
    self.losses_train_val = {"train": [], "val": []}
    self.perplexities_train_val = {"train": [], "val": []}
    self.char_perplexities_train_val = {"train": [], "val": []}
    for it in tqdm(range(max_iter)):
      if verbose and (it % eval_interval == 0 or it == max_iter - 1):
        losses, perplexity, char_perplexity = self.tokenizer.estimate_loss(self.model, EVAL_ITERS, self.model.context_window_size, self.device)
        if best_score is None:
          best_score = losses["val"]
        self.losses_train_val["train"].append(losses["train"].item())
        self.losses_train_val["val"].append(losses["val"].item())
        self.perplexities_train_val["train"].append(perplexity["train"].item())
        self.perplexities_train_val["val"].append(perplexity["val"].item())
        self.char_perplexities_train_val["train"].append(char_perplexity["train"])
        self.char_perplexities_train_val["val"].append(char_perplexity["val"])

        if early_stopping and abs(losses["val"] - best_score) < min_delta:
          print(f"Early Stopping Triggered at iteration {it}")
          break
        else:
          best_score = losses["val"]

      if verbose and (it % display_interval == 0 or it == max_iter - 1):
        print(f"iteration {it}")
        train_loss, val_loss = self.losses_train_val["train"][-1], self.losses_train_val["val"][-1]
        print(f"step {it}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")

      xb, yb = self.tokenizer.get_batch('train', self.model.context_window_size, self.device)
      logits, loss = self.model(xb, yb)
      self.loss_list.append(loss.detach().item())
      self.optimizer.zero_grad(set_to_none=True)
      loss.backward()
      self.optimizer.step()


  def save_model_data(self):
    folder_name = self.model_name

    if self.model.roe:
      folder_name += "_RoE"

    if self.model.rmsnorm:
      folder_name += "_RMSNorm"

    if self.model.gelu:
      folder_name += "_GELU"

    if self.model.dropout:
      folder_name += "_Dropout"

    if self.weight_decay != 0:
      folder_name += f"_wd{self.weight_decay}"

    os.makedirs(folder_name, exist_ok=True)
    model_path = os.path.join(folder_name, "model.pth")
    torch.save(self.model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    data = {
        "loss_list": self.loss_list,
        "losses_train_val": self.losses_train_val,
        "perplexities_train_val": self.perplexities_train_val,
        "char_perplexities_train_val": self.char_perplexities_train_val
    }
    data_path = os.path.join(folder_name, "training_data.json")
    with open(data_path, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Training data saved to {data_path}")


  def generate(self, context: str, max_new_tokens = 256) -> str:
    # given string, return out the prediction (input string included)
    context_tokens = torch.tensor(self.tokenizer.encode(context), device=self.device).reshape(1, -1)
    output_tokens = self.model.generate(context_tokens, max_new_tokens)[0].tolist()
    output_context = self.tokenizer.decode(output_tokens)
    return output_context


## Part 2 Model Training

# Test Content

In [18]:
input_text1 = """
First Citizen:
Care for us! True, indeed! They ne'er cared for us
yet: suffer us to famish, and their store-houses
crammed with grain; make edicts for usury, to
support usurers; repeal daily any wholesome act
established against the rich, and provide more
piercing statutes daily, to chain up and restrain
the poor. If the wars eat us not up, they will; and
there's all the love they bear us.
"""

target_output1 = """
MENENIUS: Either you must
Confess yourselves wondrous malicious,
Or be accused of folly. I shall tell you
A pretty tale: it may be you have heard it;
But, since it serves my purpose, I will venture
To stale 't a little more.
"""

In [19]:
input_text2 = """MIRANDA:
Why speaks my father so ungently? This
Is the third man that e'er I saw, the first
That e'er I sigh'd for: pity move my father
To be inclined my way!

FERDINAND:
O, if a virgin,
And your affection not gone forth, I'll make you
The queen of Naples.
"""

target_output2 = """PROSPERO:
Soft, sir! one word more.
They are both in either's powers; but this swift business
I must uneasy make, lest too light winning
Make the prize light.
One word more; I charge thee
That thou attend me: thou dost here usurp
The name thou owest not; and hast put thyself
Upon this island as a spy, to win it
From me, the lord on't.

FERDINAND:
No, as I am a man.

MIRANDA:
There's nothing ill can dwell in such a temple:
If the ill spirit have so fair a house,
Good things will strive to dwell with't.

PROSPERO:
Follow me.
Speak not you for him; he's a traitor. Come;
I'll manacle thy neck and feet together:
Sea-water shalt thou drink; thy food shall be
The fresh-brook muscles, wither'd roots and husks
Wherein the acorn cradled. Follow.
"""

# Baseline Model (model in Part 1)

In [24]:
charTokenizer = CharacterTokenizer(data)
baselineTransoformer = TransformerLM(charTokenizer.vocab_size, CONTEXT_WINDOW_SIZE).to(device)
baseline = ModelTokenizerSystem(baselineTransoformer, charTokenizer, device, model_name= "baseline", optimizer_weight_decay=0)

In [25]:
baseline.train()

  0%|          | 1/2000 [00:06<3:34:32,  6.44s/it]

iteration 0
step 0: train loss nan, val loss nan


  2%|▎         | 50/2000 [00:11<07:33,  4.30it/s]


KeyboardInterrupt: 

In [ ]:
baseline.save_model_data()

In [ ]:
baseline.generate(input_text1)

In [ ]:
baseline.generate(input_text2)